In [1]:
# Required modules

import os
import re
import pyts
import wave
import glob
import tqdm
import keras
import librosa
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.signal as sps
import plotly.express as px

from scipy.io import wavfile
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pyts.classification import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-10 17:53:47.646060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 17:53:48.267210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_

In [2]:
# Load the 30 sec data features

features30 = pd.read_csv("./Data/features_30_sec.csv")
features30.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
# Load the 3 sec data features

features3 = pd.read_csv("./Data/features_3_sec.csv")
features3.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [4]:
sr = 22050
new_rate = 2205
window_size = int(3 * sr)
hop_length = int(1 * sr)

def segment_audio_files(data, window_size, hop_length, labels):
    """
    Segments the audio data into fixed-size windows.
    
    :param data: Array of audio samples.
    :param window_size: The number of samples in each window.
    :param hop_length: The number of samples to slide the window by for the next segment.
    :return: A list of segmented audio windows.
    """

    segments = []
    extended_labels = []
    for sample, label in tqdm.tqdm(zip(data, labels)):
        for start in range(0, len(sample) - window_size + 1, hop_length):
            segment = sample[start:start + window_size]
            number_of_samples = round(len(segment) * float(new_rate) / sr)
            segments.append(sps.resample(segment, number_of_samples))
            extended_labels.append(label)
    return np.array(segments), extended_labels

In [5]:
all_music = list()
labels = list()

for dir_name in os.listdir("./Data/genres_original/"):
    for fname in glob.glob(f"./Data/genres_original/{dir_name}/*.wav"):
        try:
            audio_data = np.frombuffer(wave.open(fname, "r").readframes(-1)[4:], dtype=np.int16)
            number_of_samples = round(len(audio_data) * float(new_rate) / (sr*10))
            all_music.append(sps.resample(audio_data, number_of_samples))
#             all_music.append(audio_data)
            labels.append(dir_name)
        except Exception as e:
            print(fname, e)

# segmented_data, labels = segment_audio_files(all_music, window_size, hop_length, labels)

./Data/genres_original/jazz/jazz.00054.wav file does not start with RIFF id


In [6]:
# Creating train and test data

X, y = pd.DataFrame(all_music), pd.DataFrame(labels)
X.fillna(0, inplace=True)

In [7]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

/home/apeddi/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/apeddi/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [8]:
# Train and Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=88)
y_train = pd.get_dummies(y_train).astype(np.int32)
y_test = pd.get_dummies(y_test).astype(np.int32)

num_classes = 10

In [9]:
# Modifing the data

X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.Flatten()(conv3)

    output_layer = keras.layers.Dense(10, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

2024-04-10 17:55:03.772277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib
2024-04-10 17:55:03.772307: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-10 17:55:03.772369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel drive

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [11]:
# Model Training

epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    X_train,
    y_train.values,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500


ValueError: in user code:

    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/apeddi/.local/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 605, in update_

In [ ]:
model = keras.models.load_model("best_model.keras")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
# Plotting the history, which has Accuracy and Loss

def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()

    plt.show()

plot_history(history)

In [ ]:
np.unique(y_pred)

In [ ]:
# Extracting features

def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

## CNNS

In [3]:
# Create an instance of ImageDataGenerator for data augmentation (optional)

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
#     rotation_range=20,  # Data augmentation: Random rotations
#     width_shift_range=0.2,  # Data augmentation: Random horizontal shifts
#     height_shift_range=0.2,  # Data augmentation: Random vertical shifts
#     shear_range=0.2,  # Data augmentation: Random shearing
#     zoom_range=0.2,  # Data augmentation: Random zooming
#     horizontal_flip=True,  # Data augmentation: Random horizontal flips
#     fill_mode='nearest',  # Strategy for filling in newly created pixels
    validation_split=0.2  # Splitting the data: 80% for training, 20% for validation
)

# Assuming your dataset directory is "/path/to/dataset"
train_generator = train_datagen.flow_from_directory(
    './Data/images_original/',
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'  # Specify this is training data
)

validation_generator = train_datagen.flow_from_directory(
    './Data/images_original/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify this is validation data
)

Found 800 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


In [ ]:
train_imgs, valid_imgs = list(), list()
train_labels, valid_labels = list(), list()

for x, y in tqdm.tqdm(train_generator):
    train_imgs.extend(x.tolist())
    train_labels.extend(y.tolist())
print("Training Done")

for x, y in tqdm.tqdm(validation_generator):
    valid_imgs.extend(x.tolist())
    valid_labels.extend(y.tolist())

423it [03:29,  3.33it/s]                       

In [118]:
train = list()
valid = list()

for t in train_generator.filenames:
    fn = (f"{t.split('/')[1].split('.')[0][:-5]}.{t.split('/')[1].split('.')[0][-5:]}.wav")
    train.append(features30[features30["filename"] == fn].values[0][2:-1])
    
for t in validation_generator.filenames:
    fn = (f"{t.split('/')[1].split('.')[0][:-5]}.{t.split('/')[1].split('.')[0][-5:]}.wav")
    valid.append(features30[features30["filename"] == fn].values[0][2:-1])
    
train, valid = np.array(train), np.array(valid)

In [119]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, concatenate
from tensorflow.keras.layers import BatchNormalization

# Image branch
input_image = Input(shape=(150, 150, 3))
x1 = Conv2D(32, (3, 3), activation='relu')(input_image)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Conv2D(64, (3, 3), activation='relu')(x1)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Conv2D(128, (3, 3), activation='relu')(x1)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Flatten()(x1)

# Additional features branch
input_features = Input(shape=(57,))
x2 = Dense(128, activation='relu')(input_features)
x2 = Dropout(0.5)(x2)
x2 = BatchNormalization()(x2)  # Optional, helps to stabilize learning

# Merge branches
combined = concatenate([x1, x2])

# Fully connected layers
x = Dense(512, activation='relu')(combined)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)  # Assuming 10 classes

# Create the model
model = Model(inputs=[input_image, input_features], outputs=output)

In [120]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [121]:
history = model.fit(
    [train_generator, train],
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=50,
    validation_data=[validation_generator, valid],
    validation_steps=validation_generator.n // validation_generator.batch_size
)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'keras.preprocessing.image.DirectoryIterator'>", "<class 'numpy.ndarray'>"}), <class 'NoneType'>

In [89]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')

# Save the model
model.save('path/to/save/model.h5')

7/7 [==============================] - 2s 211ms/step - loss: 1.5053 - accuracy: 0.4774
Validation loss: 1.5053174495697021, Validation accuracy: 0.4773869216442108
